In [1]:
from auto_model import AutoModel
from aiohttp import web
from aiohttp.typedefs import Handler
import asyncio
import autokeras as ak
from hooks import Hook
import nest_asyncio
import pickle
import sys
from tblib import pickling_support
from typing import Any, Callable, Coroutine

nest_asyncio.apply()
pickling_support.install()

In [7]:
models: dict[str, AutoModel] = {}

In [8]:
def stream(handler: Callable[[web.Request], Coroutine[Any, Any, Any]]) -> Handler:
    async def stream(request: web.Request) -> web.StreamResponse:
        response = web.StreamResponse()
        await response.prepare(request)
        current_loop = asyncio.get_event_loop()

        def write(type: bytes):
            def write(__s: str) -> None:
                current_loop.run_until_complete(
                    response.write(type + b":::::" + bytes(__s, 'utf-8'))
                )
            return write

        with Hook(sys.stdout.write, write(b'stdout')):
            with Hook(sys.stderr.write, write(b'stderr')):
                try:
                    result = await handler(request)
                except BaseException as exception:
                    result = exception

        if isinstance(result, (asyncio.CancelledError, ConnectionResetError)):
            print("Request Cancelled ❌", file=sys.stderr)
            return response
            
        await response.write(b"result:::::" + pickle.dumps(result))
        return response

    return stream

@stream
async def get_or_create_model(request: web.Request) -> str:
    body = await request.read()
    args, kwargs = pickle.loads(body)
    model = AutoModel(*args, **kwargs)
    models[model.project_name] = model
    return model.project_name

@stream
async def get_model_attribute(request: web.Request) -> Any:
    return getattr(models[request.match_info['project_name']], request.match_info['attribute'])

@stream
async def perform_model_method(request: web.Request) -> Any:
    model = models[request.match_info['project_name']]
    args, kwargs = pickle.loads(await request.read())
    return getattr(model, request.match_info['method'])(*args, **kwargs)

@stream
async def loop(request: web.Request) -> Any:
    while True:
        try:
            await asyncio.sleep(1)
            print("Loop...")
        except BaseException as exception:
            raise exception

app = web.Application(client_max_size=1024 ** 3)
app.add_routes(
    [
        web.get("/models/{project_name}/{attribute}", get_model_attribute),
        web.post("/models/{project_name}/{method}", perform_model_method),
        web.put("/models", get_or_create_model),
        web.post("/loop", loop),
    ]
)
runner = web.AppRunner(app)
await runner.setup()

site = web.TCPSite(runner, host="0.0.0.0", port=8081)
await site.start()

while True:
    try:
        await asyncio.sleep(0.25)
    except BaseException:
        break

await site.stop()

Trial 1 Complete [00h 06m 28s]
val_loss: 0.15888012945652008

Best val_loss So Far: 0.15888012945652008
Total elapsed time: 00h 06m 28s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
adam              |adam              |optimizer
0.004             |0.00025           |learning_rate



Epoch 1/40
1273/1273 [==============================] - 15s 10ms/step - loss: 0.8391 - accuracy: 0.8297 - val_loss: 0.3698 - val_accuracy: 0.8931
Epoch 2/40
1273/1273 [==============================] - 12s 9ms/step - loss: 0.3854 - accuracy: 0.9161 - val_loss: 0.3242 - val_accuracy: 0.9089
Epoch 3/40
1273/1273 [==============================] - 12s 10ms/step - loss: 0.3731 - accuracy: 0.9205 - val_loss: 0.2965 - val_accuracy: 0.9214
Epoch 4/40
1273/1273 [==============================] - 13s 10ms/step - loss: 0.3002 - accuracy: 0.9353 - val_loss: 0.2538 - val_accuracy: 0.9322
Epoch 5/40
1273/1273 [==============================] - 12s 9ms/step - loss: 0.2939 - accuracy: 0.9385 - val_loss: 0.2566 - val_accuracy: 0.9300
Epoch 6/40
1273/1273 [==============================] - 14s 11ms/step - loss: 0.2881 - accuracy: 0.9408 - val_loss: 0.3136 - val_accuracy: 0.9244
Epoch 7/40
1273/1273 [==============================] - 12s 9ms/step - loss: 0.2811 - accuracy: 0.9423 - val_loss: 0.2338 - va

Request Cancelled ❌
